# EPM Creep Response

**Elasto-Plastic Model (EPM) — Creep under constant stress**

## Learning Objectives

- Understand bounded vs unbounded creep in EPM
- Fit real biological (mucus) creep data using NLSQ
- Estimate yield stress from creep behavior
- Cross-validate with flow curve parameters

## Prerequisites

- Complete `01_epm_flow_curve.ipynb` for EPM basics
- Understanding of creep compliance J(t) measurements

## Estimated Runtime

- Fast demo (1 chain): ~3-4 min
- Full run (4 chains): ~8-12 min

## 1. Setup & Imports

In [ ]:
# Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
%matplotlib inline
import time
import sys
import os
import json

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.core.data import RheoData
from rheojax.models.epm.lattice import LatticeEPM

jax, jnp = safe_import_jax()
verify_float64()

FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
print(f"FAST_MODE: {FAST_MODE}")

def compute_fit_quality(y_true, y_pred):
    """Compute R2 and RMSE."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    residuals = y_true - y_pred
    if y_true.ndim > 1:
        residuals = residuals.ravel()
        y_true = y_true.ravel()
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0
    rmse = np.sqrt(np.mean(residuals**2))
    return {"R2": r2, "RMSE": rmse}

## 2. Theory: Creep in EPM

In creep, we apply a constant stress σ₀ and monitor the resulting strain γ(t). The creep compliance is:

$$J(t) = \frac{\gamma(t)}{\sigma_0}$$

### Bounded vs Unbounded Creep

EPM exhibits different behavior depending on the applied stress relative to the yield threshold:

**Bounded creep (σ₀ < σ_c,mean):**
- Stress is below most yield thresholds
- Only sites with low thresholds yield occasionally
- Strain approaches a plateau: γ(t) → γ_∞
- Material behaves as a solid

**Unbounded creep (σ₀ > σ_c,mean):**
- Stress exceeds most yield thresholds
- Continuous plastic flow
- Strain grows indefinitely: γ(t) ~ t at long times
- Material flows like a viscoplastic fluid

### Yield Stress Estimation

The crossover from bounded to unbounded creep occurs near σ_c,mean:

$$\sigma_y \approx \sigma_{c,\text{mean}} \pm \sigma_{c,\text{std}}$$

By fitting creep data at different stress levels, we can estimate the yield stress.

## 3. Load Creep Data

We use creep compliance data from mucus, a biological viscoelastic material.

In [ ]:
data_path = os.path.join("..", "data", "creep", "biological", "creep_mucus_data.csv")
if IN_COLAB:
    data_path = "creep_mucus_data.csv"
    if not os.path.exists(data_path):
        print("Please upload creep_mucus_data.csv or adjust the path.")

# Load data (tab-separated: Time, Creep Compliance)
raw = np.loadtxt(data_path, delimiter="\t", skiprows=1)
time_data = raw[:, 0]      # Time [s]
J_data = raw[:, 1]         # Creep compliance [1/Pa]

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2f} – {time_data.max():.1f} s")
print(f"J range: {J_data.min():.4f} – {J_data.max():.4f} 1/Pa")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Linear scale
ax1.plot(time_data, J_data, "ko-", markersize=5)
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Creep compliance J(t) [1/Pa]")
ax1.set_title("Creep Compliance: Mucus")
ax1.grid(True, alpha=0.3)

# Right: Log-log scale
ax2.loglog(time_data, J_data, "ko-", markersize=5)
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Creep compliance J(t) [1/Pa]")
ax2.set_title("Log-Log Plot")
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

The creep compliance shows:
- Rapid initial increase (elastic + fast relaxation)
- Gradual approach to plateau (bounded creep)
- This suggests the applied stress is below the yield stress

## 4. NLSQ Fitting

For creep, EPM predicts strain γ(t) at constant applied stress. We need to specify the applied stress level.

In [ ]:
# Estimate applied stress from plateau compliance
# J_∞ ≈ 1/G_∞ → σ₀ typically small for biological materials
# For mucus, typical stress ~ 0.1-1 Pa

SIGMA_0 = 0.5  # Applied stress [Pa] - typical for mucus rheometry

# Convert compliance to strain
gamma_data = J_data * SIGMA_0

print(f"Applied stress σ₀ = {SIGMA_0} Pa")
print(f"Strain range: {gamma_data.min():.4f} – {gamma_data.max():.4f}")

In [ ]:
# Initialize LatticeEPM
model = LatticeEPM(
    L=16 if FAST_MODE else 32,
    dt=0.01,
    mu=0.5,
    tau_pl=1.0,
    sigma_c_mean=1.0,
    sigma_c_std=0.1,
)

# Set bounds appropriate for biological materials
model.parameters["mu"].bounds = (0.01, 10.0)
model.parameters["tau_pl"].bounds = (0.1, 100.0)
model.parameters["sigma_c_mean"].bounds = (0.1, 10.0)
model.parameters["sigma_c_std"].bounds = (0.01, 2.0)

print("LatticeEPM initialized for creep fitting")

In [ ]:
# Fit to creep data
print("Fitting to creep data...")
t0 = time.time()
model.fit(time_data, gamma_data, test_mode="creep", stress=SIGMA_0, method='scipy')
t_nlsq = time.time() - t0

# Compute fit quality
y_pred = model.predict(time_data, test_mode="creep", stress=SIGMA_0, smooth=True).y
metrics = compute_fit_quality(gamma_data, y_pred)

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"R²: {metrics['R2']:.6f}")
print(f"RMSE: {metrics['RMSE']:.6f}")

print("\nFitted parameters:")
param_names = ["mu", "tau_pl", "sigma_c_mean", "sigma_c_std"]
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:15s} = {val:.4g}")

In [ ]:
# Predict and plot (compliance J(t) = gamma(t) / sigma_0)
param_names = ["mu", "tau_pl", "sigma_c_mean", "sigma_c_std"]

time_fine = np.linspace(time_data.min(), time_data.max() * 1.2, 200)
rheo_fine = RheoData(
    x=time_fine, y=np.zeros_like(time_fine),
    initial_test_mode="creep",
    metadata={"stress": SIGMA_0},
)

gamma_pred = model.predict(rheo_fine, smooth=True).y
J_pred = np.array(gamma_pred) / SIGMA_0

fig, ax = plt.subplots(figsize=(9, 6))
ax.plot(time_data, J_data, "ko", markersize=6, label="Data")
ax.plot(time_fine, J_pred, "-", lw=2, color="C0", label="EPM fit")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Creep compliance J(t) [1/Pa]")
ax.set_title(f"Creep Fit (R2 = {metrics['R2']:.4f})")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Yield Stress Estimation

From the fitted σ_c,mean, we can estimate the yield stress.

In [ ]:
sigma_c_mean = model.parameters.get_value("sigma_c_mean")
sigma_c_std = model.parameters.get_value("sigma_c_std")

print("Yield Stress Estimation")
print("=" * 40)
print(f"σ_c,mean = {sigma_c_mean:.4f} Pa")
print(f"σ_c,std  = {sigma_c_std:.4f} Pa")
print(f"\nEstimated yield stress:")
print(f"  σ_y ≈ {sigma_c_mean:.3f} ± {sigma_c_std:.3f} Pa")
print(f"\nApplied stress σ₀ = {SIGMA_0:.3f} Pa")

if SIGMA_0 < sigma_c_mean - sigma_c_std:
    print(f"\nσ₀ < σ_y: Bounded creep (solid-like) - consistent with plateau behavior")
elif SIGMA_0 > sigma_c_mean + sigma_c_std:
    print(f"\nσ₀ > σ_y: Unbounded creep (fluid-like) - expect continued flow")
else:
    print(f"\nσ₀ ≈ σ_y: Near yield point - mixed behavior")

## 6. Bayesian Inference

In [ ]:
# Warm-start from NLSQ
initial_values = {
    name: model.parameters.get_value(name)
    for name in model.parameters.keys()
}

# --- Fast demo config ---
NUM_WARMUP = 50 if FAST_MODE else 200
NUM_SAMPLES = 100 if FAST_MODE else 500
NUM_CHAINS = 1

print(f"Running Bayesian inference: {NUM_CHAINS} chain(s)")

t0 = time.time()
# For creep mode, store stress on model before Bayesian fit
model._cached_stress = SIGMA_0

result = model.fit_bayesian(
    time_data,
    gamma_data,
    test_mode="creep",
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"Bayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence diagnostics
diag = result.diagnostics

print("Convergence Diagnostics")
print("=" * 50)
print(f"{'Parameter':>15s}  {'R-hat':>8s}  {'ESS':>8s}")
print("-" * 50)
for p in param_names:
    r_hat = diag.get("r_hat", {}).get(p, float("nan"))
    ess = diag.get("ess", {}).get(p, float("nan"))
    print(f"{p:>15s}  {r_hat:8.4f}  {ess:8.0f}")

n_div = diag.get("divergences", diag.get("num_divergences", 0))
print(f"\nDivergences: {n_div}")

In [ ]:
# ArviZ diagnostic plots
display_arviz_diagnostics(result, param_names, fast_mode=FAST_MODE)

## 7. Posterior Predictive Check

In [ ]:
posterior = result.posterior_samples
n_draws = min(5 if FAST_MODE else 100, len(list(posterior.values())[0]))

print(f"Computing {n_draws} posterior predictive samples...")
pred_samples = []
for i in range(n_draws):
    for name in param_names:
        model.parameters.set_value(name, float(posterior[name][i]))
    pred_i = model.predict(rheo_fine, smooth=True).y
    J_i = np.array(pred_i) / SIGMA_0
    pred_samples.append(J_i)

pred_samples = np.array(pred_samples)
pred_median = np.median(pred_samples, axis=0)
pred_lo = np.percentile(pred_samples, 2.5, axis=0)
pred_hi = np.percentile(pred_samples, 97.5, axis=0)
print("Done.")

fig, ax = plt.subplots(figsize=(9, 6))
ax.fill_between(time_fine, pred_lo, pred_hi, alpha=0.3, color="C0", label="95% CI")
ax.plot(time_fine, pred_median, "-", lw=2, color="C0", label="Posterior median")
ax.plot(time_data, J_data, "ko", markersize=6, label="Data")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Creep compliance J(t) [1/Pa]")
ax.set_title("Posterior Predictive Check - Creep")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 8. Yield Stress Posterior

In [ ]:
# Posterior distribution of yield stress
sigma_c_mean_post = np.array(posterior["sigma_c_mean"])
sigma_c_std_post = np.array(posterior["sigma_c_std"])

# Yield stress estimate
sigma_y_post = sigma_c_mean_post  # Central estimate

fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(sigma_y_post, bins=30, density=True, alpha=0.7, color="C0", edgecolor="black")
ax.axvline(np.median(sigma_y_post), color="C1", linestyle="--", lw=2, 
           label=f"Median: {np.median(sigma_y_post):.3f} Pa")
ax.axvline(SIGMA_0, color="red", linestyle=":", lw=2, 
           label=f"Applied stress: {SIGMA_0} Pa")

ax.set_xlabel("Yield stress σ_y [Pa]")
ax.set_ylabel("Probability density")
ax.set_title("Posterior Distribution of Yield Stress")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

# Summary
print("\nYield Stress Posterior")
print("=" * 40)
print(f"Median: {np.median(sigma_y_post):.4f} Pa")
print(f"95% CI: [{np.percentile(sigma_y_post, 2.5):.4f}, {np.percentile(sigma_y_post, 97.5):.4f}] Pa")

## 9. Parameter Summary

In [ ]:
print("\nParameter Summary")
print("=" * 65)
print(f"{'Param':>15s}  {'Median':>12s}  {'95% CI':>28s}")
print("-" * 65)

for name in param_names:
    samples = posterior[name]
    median = float(np.median(samples))
    lo = float(np.percentile(samples, 2.5))
    hi = float(np.percentile(samples, 97.5))
    print(f"{name:>15s}  {median:12.4g}  [{lo:.4g}, {hi:.4g}]")

## 10. Key Takeaways

1. **EPM captures creep physics** through the interplay of applied stress and yield threshold distribution
2. **Bounded creep** (plateau behavior) indicates σ₀ < σ_y — the material is below yield
3. **σ_c,mean directly estimates yield stress** from creep data
4. **Biological materials** like mucus show complex viscoelastic behavior well-captured by EPM
5. **Bayesian inference** provides uncertainty on yield stress estimates

## Cross-Validation

To validate these results:
- Compare σ_c,mean from creep with flow curve estimates (Notebook 01)
- Perform creep at multiple stress levels to map the yield transition
- Check consistency between SAOS (G') and creep (J_∞ ≈ 1/G_∞)

## Next Steps

- **Notebook 05**: Stress relaxation and relaxation spectrum
- **Notebook 06**: Visualization of lattice stress fields during creep

In [ ]:
# Save results
output_dir = os.path.join("..", "outputs", "epm", "creep")
os.makedirs(output_dir, exist_ok=True)

median_params = {name: float(np.median(posterior[name])) for name in param_names}
with open(os.path.join(output_dir, "nlsq_params_creep.json"), "w") as f:
    json.dump(median_params, f, indent=2)

print(f"Results saved to {output_dir}/")